In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re

%matplotlib inline

In [2]:
df_train = pd.read_csv('input/train.csv')
# df = df[pd.notnull(df['tags'])]
df_train.head(10)

,itemid,title,Category,image_path
0,307504,nyx sex bomb pallete natural palette,0,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg
1,461203,etude house precious mineral any cushion pearl...,1,beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg
2,3592295,milani rose powder blush,2,beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg
3,4460167,etude house baby sweet sugar powder,3,beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg
4,5853995,bedak revlon color stay aqua mineral make up,3,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg
5,6208490,dr pure whitening cream,4,beauty_image/77e6b7e9d5544adbfda6809b2351c4fa.jpg
6,6959546,chanel powder blush malice,2,beauty_image/485c4b8435a1e94976a569f3f014ea8b.jpg
7,8984461,snail white cream original 100,4,beauty_image/c7336f1c2e590d4c2bee219a6a0351a7.jpg
8,9184082,sunprise all proof spf 50,4,beauty_image/970b32aa659689f371516e5ceb423e25.jpg
9,10345338,eyebrow powder nyx satuan rp 15.000 pc,3,beauty_image/49a47745248031775c98593f70f7201a.jpg


In [30]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, TimeDistributed, Embedding
from keras.preprocessing import text, sequence
from keras import utils

In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[]

In [5]:
train_posts, test_posts, train_tags, test_tags = train_test_split(df_train['title'], 
                                                                  df_train['Category'], 
                                                                  test_size=0.33, 
                                                                  random_state=42)

In [64]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [65]:
tokenize.fit_on_texts(train_posts) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [66]:
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [67]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [68]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (446632, 1000)
x_test shape: (219983, 1000)
y_train shape: (446632, 58)
y_test shape: (219983, 58)


In [79]:
batch_size = 256
epochs = 10

In [70]:
x_train[1,:]

array([0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [71]:
train_posts

631739    promo cuci gudang oppo a7 64gb kredit tampa ca...
501446    baju cewek wanita kaos v neck lengan panjang p...
217843                   bb cream missha 20 ml spf 42 no 21
25423                            acne scar cream cheapbrand
337588    dress sexy bahan lace rajut model hollow untuk...
648117    kualitas terbaik charger asus 2 a 9v fast char...
218155                      the body shop tea tree bb cream
45935                  ertos why acne night treatment cream
624847    xperia z5premium e6853 single chrome ram3 32gb...
471137    kaos t shirt wanita casual lengan pendek model...
92540                 facial wash for normal skin theraskin
415487    pakaian wanita baju blouse cewek kerah v biru ...
129367    share 30 ml cosrx low ph good morning gel clea...
39356     design modern etudehouse etude house dear girl...
306089    gaun wanita model ikatan tali dengan kerah hal...
297200    dress rajut mini slim turtleneck lengan panjan...
351100    dress a line v neck lengan 3 4

In [72]:
# Build the model
model = Sequential()
model.add(LSTM(120,input_shape=(1,max_words)))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))
model.add(Dense(num_classes, activation='relu'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [80]:
history = model.fit(x_train.reshape((x_train.shape[0],1,x_train.shape[1])), y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 401968 samples, validate on 44664 samples
Epoch 1/10
401968/401968 [==============================] - 114s 285us/step - loss: 1.4237 - acc: 0.6408 - val_loss: 1.4391 - val_acc: 0.6372
Epoch 2/10
401968/401968 [==============================] - 118s 294us/step - loss: 1.3977 - acc: 0.6463 - val_loss: 1.4276 - val_acc: 0.6395: 1.3985 
Epoch 3/10
401968/401968 [==============================] - 113s 282us/step - loss: 1.3746 - acc: 0.6513 - val_loss: 1.4187 - val_acc: 0.6419
Epoch 4/10
401968/401968 [==============================] - 138s 343us/step - loss: 1.3533 - acc: 0.6571 - val_loss: 1.4143 - val_acc: 0.6432
Epoch 5/10
401968/401968 [==============================] - 128s 319us/step - loss: 1.3335 - acc: 0.6613 - val_loss: 1.4118 - val_acc: 0.6436
Epoch 6/10
401968/401968 [==============================] - 120s 299us/step - loss: 1.3146 - acc: 0.6660 - val_loss: 1.4093 - val_acc: 0.6455
Epoch 7/10
401968/401968 [==============================] - 122s 304us/step - loss: 1.29

In [81]:
score = model.evaluate(x_test.reshape((x_test.shape[0],1,x_test.shape[1])), y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

219983/219983 [==============================] - 21s 95us/step
Test accuracy: 0.6455544292142926


In [82]:
df_public = pd.read_csv('input/test.csv')
df_public.head(10)

,itemid,title,image_path
0,370855998,flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg
1,637234604,maybelline clear smooth all in one bb cream sp...,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg
2,690282890,murah innisfree eco natural green tea bb cream...,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg
3,930913462,loreal white perfect day cream spf 17 pa white...,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg
4,1039280071,hada labo cc cream ultimate anti aging spf 35 ...,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg
5,1327710392,cathy doll cc speed white powder pact spf 40 o...,beauty_image/e1e50828d5594721a7d5d5c1ff78afbd.jpg
6,1328802799,safi white natural brightening cream 45g,beauty_image/97ec852d5afc5d82ac02b80083cf292f.jpg
7,1330468145,light beige 03 bioaqua bb cushion exquisite de...,beauty_image/8ce1a5fe546f0cc795329bad599a8d5a.jpg
8,1677309730,new produk missha m perfect bb cream share in ...,beauty_image/755fcc85c687e8cb53d2a8d43ebfe251.jpg
9,1683142205,ready laneige bb cushion anti aging spf 50 pa,beauty_image/34b56398c099505c650cf2447dc9f21f.jpg


In [83]:
x_public = tokenize.texts_to_matrix(df_public["title"])

In [84]:
preds = model.predict(x_public.reshape((x_public.shape[0],1,x_public.shape[1])), verbose=1)

172402/172402 [==============================] - 30s 174us/step


In [85]:
df_public['Category'] = [np.argmax(pred) for pred in preds]
df_submit = df_public[['itemid', 'Category']].copy()
df_submit.to_csv('submission.csv', index=False)